# Scraper les informations d'une référence parfum sur Fragrantica

## Imports

In [3]:
from bs4 import BeautifulSoup
import re
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import numpy as np

## Les fonctions

### Fonction pour récuperer le code html d'une page parfum

In [2]:
#Récupère le HTML de la page correspondant à l'URL donnée.
def get_html_non_headless(url):
    """
    Ouvre Chrome en mode normal (non-headless), va sur l'URL, 
    attend 1 seconde, récupère le HTML et ferme le navigateur.
    """
    # 1) Configuration basique du driver, sans headless
    options = webdriver.ChromeOptions()
    # Pas d'options.add_argument('--headless'), car on veut voir la fenêtre

    # 2) Création du driver
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    
    # 3) Accès à la page
    driver.get(url)
    
    # 4) Patiente un peu pour laisser la page (et scripts éventuels) se charger
    time.sleep(1)
    
    # 5) Récupère le code HTML
    html_content = driver.page_source
    
    # 6) Ferme le navigateur
    driver.quit()

    return html_content

### Fonctions pour récuperer les infos sur un parfum

In [8]:
# 1) Nom du parfum
def extract_perfume_name(html_content):
    """Extrait le nom du parfum depuis le contenu HTML"""
    match = re.search(r'/perfume/[^/]+/([^/]+)-\d+\.html', html_content)
    return match.group(1).replace('-', ' ') if match else None

# 2) Marque
def extract_brand_name(html_content):
    """Extrait la marque du parfum depuis le contenu HTML"""
    match = re.search(r'/perfume/([^/]+)/', html_content)
    return match.group(1).replace('-', ' ') if match else None

#3) Parfumeur
def extract_nose(soup):
    """Extrait le nom du parfumeur"""
    nose_el = soup.select_one('div.cell a[href^="/noses/"]')
    return nose_el.get_text(strip=True) if nose_el else None

#4) Année de sortie
def extract_launch_year(soup):
    """Extrait l'année de sortie du parfum"""
    title_el = soup.find('title')
    if title_el:
        possible_year = title_el.get_text(strip=True)[-4:]
        if possible_year.isdigit():
            return possible_year
    return None

#5) Perfume rating
def extract_rating(soup):
    """Extrait la note du parfum"""
    rating_el = soup.select_one('span[itemprop="ratingValue"]')
    return rating_el.get_text(strip=True) if rating_el else None

#6) Nombre de votes
def extract_rating_count(soup):
    """Extrait le nombre de votes"""
    rating_count_el = soup.select_one('span[itemprop="ratingCount"]')
    return rating_count_el.get_text(strip=True) if rating_count_el else None

#7) Accords principaux
def extract_main_accords(soup):
    """Extrait les accords principaux du parfum"""
    main_accords_el = soup.find_all('div', class_='cell accord-box')
    return [element.get_text(strip=True) for element in main_accords_el if element.get_text(strip=True)]

#8) Genre
def extract_gender(gender_list):
    """Détermine le genre du parfum en fonction des votes"""
    if not gender_list:
        return None
    
    sum_votes = np.sum([int(element[1]) for element in gender_list])
    
    if sum_votes > 8:
        female_count = int(gender_list[0][1]) + int(gender_list[1][1])
        male_count = int(gender_list[3][1]) + int(gender_list[4][1])
        unisex_count = 1.2 * int(gender_list[2][1])

        if female_count > male_count and female_count > unisex_count:
            return "female"
        elif male_count > unisex_count:
            return "male"
        else:
            return "unisex"
    return None

#9) Longévité
def extract_longevity(longevity_list):
    """Détermine la longévité du parfum"""
    if not longevity_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in longevity_list])
    if sum_votes > 8:
        longevity = np.argmax([int(element[1]) for element in longevity_list])
        return longevity_list[longevity][0]
    return None

#10) Sillage
def extract_sillage(sillage_list):
    """Détermine le sillage du parfum"""
    if not sillage_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in sillage_list])
    if sum_votes > 8:
        sillage = np.argmax([int(element[1]) for element in sillage_list])
        return sillage_list[sillage][0]
    return None

#11) Ressenti du prix
def extract_price_feeling(price_feeling_list):
    """Détermine le ressenti du prix du parfum"""
    if not price_feeling_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in price_feeling_list])
    if sum_votes > 8:
        price_feeling = np.argmax([int(element[1]) for element in price_feeling_list])
        return price_feeling_list[price_feeling][0]
    return None

#12) Pyramide olfactive

### Fonctions intermediaire pour le tableau avec Gender, Sillage, loongevity et Price feeling

In [5]:
def extract_votes(soup):
    """Extrait les catégories et leurs votes à partir du HTML."""
    tableau_votes = []
    tab_rows = soup.select('div.grid-x.grid-margin-x')  # Sélection des blocs contenant les votes
    
    for row in tab_rows:
        category_el = row.select_one('span.vote-button-name')  # Nom de la catégorie
        votes_el = row.select_one('span.vote-button-legend')  # Nombre de votes

        if category_el and votes_el:
            category = category_el.get_text(strip=True)
            votes = votes_el.get_text(strip=True)

            # Vérifier si le nombre de votes est bien un chiffre
            if votes.isdigit():
                tableau_votes.append((category, votes))

    return tableau_votes

def filter_votes(tableau_votes, labels, remove_first=False, remove_last=False):
    """
    Filtre les votes selon un ensemble de labels et enlève potentiellement le premier ou dernier élément.
    
    :param tableau_votes: Liste des votes [(catégorie, nombre)]
    :param labels: Ensemble des labels à filtrer
    :param remove_first: Supprime le premier élément si True
    :param remove_last: Supprime le dernier élément si True
    :return: Liste filtrée et éventuellement modifiée
    """
    filtered_list = [element for element in tableau_votes if element[0] in labels]

    if remove_first and filtered_list:
        filtered_list.pop(0)
    if remove_last and filtered_list:
        filtered_list.pop()
    
    return filtered_list

## Le script

### Obtenir le contenu html de la page d'Angel Share

In [6]:
url = "https://www.fragrantica.com/perfume/By-Kilian/Angels-Share-62615.html"
html_content = get_html_non_headless(url)
    
# Vous pouvez ensuite parser le HTML avec BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')


### Mettre les variables d'un parfum dans un dico 'data'

In [11]:
data = {}

tableau_votes = extract_votes(soup)

# Définition des groupes
longevity_labels = {"very weak", "weak", "moderate", "long lasting", "eternal"}
sillage_labels = {"intimate", "moderate", "strong", "enormous"}
gender_labels = {"female", "more female", "unisex", "more male", "male"}
price_feeling_labels = {"way overpriced", "overpriced", "ok", "good value", "great value"}

# Séparation des données avec gestion des exclusions spécifiques
longevity_list = filter_votes(tableau_votes, longevity_labels, remove_last=True)  # Supprime le dernier élément
sillage_list = filter_votes(tableau_votes, sillage_labels, remove_first=True)  # Supprime le premier élément
gender_list = filter_votes(tableau_votes, gender_labels)
price_feeling_list = filter_votes(tableau_votes, price_feeling_labels)

data = {
    "nom_parfum": extract_perfume_name(html_content),
    "marque": extract_brand_name(html_content),
    "nose": extract_nose(soup),
    "launch_year": extract_launch_year(soup),
    "rating_value": extract_rating(soup),
    "rating_count": extract_rating_count(soup),
    "main_accords": extract_main_accords(soup),
    "gender": extract_gender(gender_list),
    "longevity": extract_longevity(longevity_list),
    "sillage": extract_sillage(sillage_list),
    "price_feeling": extract_price_feeling(price_feeling_list),
}

print(data)

{'nom_parfum': 'Angels Share', 'marque': 'By Kilian', 'nose': 'Benoist Lapouza', 'launch_year': '2020', 'rating_value': '4.38', 'rating_count': '15,727', 'main_accords': ['woody', 'warm spicy', 'sweet', 'vanilla', 'cinnamon', 'amber', 'powdery'], 'gender': 'unisex', 'longevity': 'long lasting', 'sillage': 'moderate', 'price_feeling': 'overpriced'}
